In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
class RecipeRecommender:
    
    def __init__(self, ingreditans = [], sıradaki_besli = [0,5],):
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.sırdaki_besli = sıradaki_besli
        self.ingreditans = ingreditans if ingreditans is not None else []
        self.df = pd.read_csv("RAW_recipes.csv")
    
    def benzer_yemekler(self):
        df = pd.read_csv("RAW_recipes.csv")
        tfidf = TfidfVectorizer()
        malzemeler_str = " ".join(self.ingreditans)
        malzemeler_vektoru = tfidf.fit_transform([malzemeler_str])
        cosbenzerlik = cosine_similarity(malzemeler_vektoru, tfidf.transform(self.df["ingredients"]))
        self.df["cosbenzerlik"] = cosbenzerlik[0]
        self.df = df.sort_values(by="cosbenzerlik", ascending=False)
        
    def sıradaki_yemekler(self):
        benzer_yemekler = df[self.sırdaki_besli[0]:self.sırdaki_besli[1]]
        while True:
            devam_mı = input("çıkış için : q, sırdaki beşli için 1")
            if devam_mı == 1:
                self.sırdaki_besli[0] +=5
                self.sırdaki_besli[1] +=5
                return benzer_yemekler
            else:
                break
    def içindekiler(self,yeni_yemek):
        self.ingreditans.append(yeni_yemek)

yemek = RecipeRecommender()
while True:
    islem = input("işlem seçiniz")
    if islem == "q":
        print("işlem sonlandırılıyore...")
        break
    elif islem == "1":
        yemek.içindekiler()
    elif islem == "4":
        yemek_ismi = input("yemek ismi giriniz")
        yemek_listesi = yemek_ismi.split(",")
        for i in yemek_listesi:
            yemek.içindekiler(i)


## ANA KOD

In [ ]:
import sqlite3

# veritabanı bağlantısı oluştur
conn = sqlite3.connect('yemekler.db')

# yemekler tablosunu oluştur
conn.execute('''CREATE TABLE yemekler
                (id INTEGER PRIMARY KEY,
                isim TEXT NOT NULL,
                malzemeler TEXT NOT NULL,
                tarif TEXT NOT NULL);''')

# veritabanını kaydet ve bağlantıyı kapat
conn.commit()
conn.close()

In [ ]:
import sqlite3

# veritabanı bağlantısı oluştur
conn = sqlite3.connect('yemekler.db')

# kullanıcının malzemelerini girdiği bir liste oluştur
malzemeler = []
while True:
    malzeme = input("Elimizde hangi malzemeler var? (Çıkmak için 'q' tuşuna basın)\n")
    if malzeme == 'q':
        break
    malzemeler.append(malzeme)

# uygun tarifleri seç
cur = conn.cursor()
cur.execute("SELECT isim, malzemeler, tarif FROM yemekler")
rows = cur.fetchall()

uygun_tarifler = []
for row in rows:
    tarif_malzemeleri = row[1].split(", ")
    if set(tarif_malzemeleri).issubset(malzemeler):
        uygun_tarifler.append(row)

# uygun tarifleri ekrana yazdır
if uygun_tarifler:
    for tarif in uygun_tarifler:
        print("Tarif Adı:", tarif[0])
        print("Malzemeler:", tarif[1])
        print("Tarif:", tarif[2])
        print()
else:
    print("Elimizdeki malzemelerle uygun bir tarif bulunamadı.")

# veritabanı bağlantısını kapat
conn.close()


## YEMEK EKLEME

In [ ]:

import sqlite3

# veritabanı bağlantısı oluştur
conn = sqlite3.connect('yemekler.db')

# kullanıcıdan yeni yemek bilgilerini al
isim = input("Yeni yemek adı: ")
malzemeler = input("Malzemeler (virgülle ayrılmış): ")
tarif = input("Tarif: ")

# veritabanına yeni yemek kaydı ekle
conn.execute("INSERT INTO yemekler (isim, malzemeler, tarif) VALUES (?, ?, ?)",
             (isim, malzemeler, tarif))

# veritabanını kaydet ve bağlantıyı kapat
conn.commit()
conn.close()

## YEMEK SİLME


In [ ]:
import sqlite3

# veritabanı bağlantısı oluştur
conn = sqlite3.connect('yemekler.db')

# kullanıcıdan silinecek yemek adını al
silinacak_yemek = input("Silmek istediğiniz yemeğin adı: ")

# yemekleri seç ve kullanıcının girdiği yemek adını içeren kayıtları bul
cur = conn.cursor()
cur.execute("SELECT * FROM yemekler WHERE isim = ?", (silinacak_yemek,))
rows = cur.fetchall()

if len(rows) == 0:
    print("Böyle bir yemek kaydı bulunamadı.")
else:
    # bulunan kaydı sil
    conn.execute("DELETE FROM yemekler WHERE isim = ?", (silinacak_yemek,))
    print(f"{len(rows)} adet yemek kaydı silindi.")

# veritabanını kaydet ve bağlantıyı kapat
conn.commit()
conn.close()
